In [ ]:
# Deep learning
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# Read Dataset
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# Miscelanous
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Dataset Path
TRAIN_PATH = "/kaggle/input/intel-image-classification/seg_train/seg_train"
TEST_PATH = "/kaggle/input/intel-image-classification/seg_test/seg_test"
PRED_PATH = "/kaggle/input/intel-image-classification/seg_pred/seg_pred"

In [ ]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Preparation

In [ ]:
label_mapping = {
    'buildings': 0,
    'forest':1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5 
}

In [ ]:
for category in os.listdir(TRAIN_PATH):
    category_dir = os.path.join(TRAIN_PATH, category)
    print(category_dir)

## Read dataset with process bar from tqdm

In [ ]:
def read_data(path):
    X = []
    y = []
    for category in os.listdir(path):
        category_dir = os.path.join(path, category)
        for img_file in tqdm(os.listdir(category_dir)):
            img_path = os.path.join(category_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (150,150))
            # convert img to torch tensor
            transform = transforms.ToTensor()
            img = transform(img)
            X.append(img)
            y.append(label_mapping[category])
            
    y = torch.tensor(y)
    return (X,y)

In [ ]:
X_train, y_train = read_data(TRAIN_PATH)

## Custom Dataset

In [ ]:
class IntelDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.images = X
        self.labels = y
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx],self.labels[idx]

In [ ]:
train_dataset = IntelDataset(X_train, y_train)

## Data Loader

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

# Modeling

In [ ]:
# 3x3 convolution layer
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [ ]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=6):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(1024, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Training

In [ ]:
for epoch in range(10):
    model.train()
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backpropagation and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%110==0:
            print (f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Testing

In [ ]:
X_test, y_test = read_data(TEST_PATH)

In [ ]:
test_dataset = IntelDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # predict
        preds = model(images)
        # get the label
        _, predicted = torch.max(preds.data, 1)
        # num of images
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy of the model on the test images: {100 * correct / total}%')